# 1. Generate Network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import logging
import numpy as np
import torch.nn as nn


# #########################################################################
# 1. Base Net
# #########################################################################
class BaseNet(nn.Module):
    """
    Base class for all neural networks.
    """

    def __init__(self):
        super().__init__()
        self.logger = logging.getLogger(self.__class__.__name__)
        self.rep_dim = None

    def forward(self, *input):
        raise NotImplementedError

    def summary(self):
        net_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in net_parameters])
        self.logger.info('Trainable parameters: {}'.format(params))
        self.logger.info(self)


In [3]:
# #########################################################################
# 1. Encoder
# #########################################################################
class Guassian3DNet(BaseNet):
    def __init__(self, rep_dim=2):
        super().__init__()

        self.rep_dim = rep_dim
        self.fc1 = nn.Linear(3, 2, bias=True)
        self.fc2 = nn.Linear(2, self.rep_dim, bias=True)

    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        return x

In [4]:
# #########################################################################
# 2. Decoder
# #########################################################################
class Guassian3DNetDecoder(BaseNet):
    def __init__(self, rep_dim=2):
        super().__init__()

        self.rep_dim = rep_dim
        self.fc1 = nn.Linear(self.rep_dim, 2, bias=True)
        self.fc2 = nn.Linear(2, 3, bias=True)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        return x

In [5]:
# #########################################################################
# 3. Autoencoder
# #########################################################################
class Gaussian3DNetAutoencoder(BaseNet):
    def __init__(self, rep_dim=2):
        super().__init__()

        self.rep_dim = rep_dim
        self.encoder = Guassian3DNet(rep_dim=rep_dim)
        self.decoder = Guassian3DNetDecoder(rep_dim=rep_dim)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
net = Gaussian3DNetAutoencoder()

# 2. Generate Datasete

In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [8]:
cov = [[0.1, 0, 0], 
       [0, 0.1, 0], 
       [0, 0, 0.1]]

mu_0 = [1, 1, 1]
mu_1 = [1, - 1, 1]

In [9]:
X = np.random.multivariate_normal(mu, cov, 5000)

In [10]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

In [12]:
X_ = net(torch.tensor(X, dtype=torch.float32))

In [13]:
X_

tensor([[-0.5826,  0.3281,  0.7432],
        [-0.6004,  0.3367,  0.7540],
        [-0.6057,  0.3393,  0.7573],
        ...,
        [-0.6017,  0.3373,  0.7548],
        [-0.6111,  0.3419,  0.7606],
        [-0.5721,  0.3230,  0.7368]], grad_fn=<AddmmBackward>)